In [1]:
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

In [3]:
# Lista de nombres de columnas
columns = ['NUMERO_IDENTIFICADOR', 'NOMBRE_ESTANDARIZADO', 'MARCA_ESTANDARIZADA']

# Cargar el archivo CSV con los nombres de columnas correctos
df_estandarizado = None
df_estandarizado = pd.read_csv("https://deinsoluciones.cl/web_semantica/para_end_point_2.csv", delimiter=",", encoding='utf-8', header=None, names=columns,skiprows=1)

# Ver los primeros registros
df_estandarizado.head()

,NUMERO_IDENTIFICADOR,NOMBRE_ESTANDARIZADO,MARCA_ESTANDARIZADA
0,20997043,CAMION,RAM
1,20955958,BUS,GOLDEN DRAGON
2,20969631,BUS,MITSUBISHI FUSO
3,20987696,BUS,MITSUBISHI FUSO
4,20961954,AUTOMOVIL,NISSAN


In [4]:
df_estandarizado_unique = df_estandarizado[['NUMERO_IDENTIFICADOR', 'NOMBRE_ESTANDARIZADO', 'MARCA_ESTANDARIZADA']].drop_duplicates()

In [5]:
# Agregar la columna ID como un índice único
df_estandarizado_unique.insert(0, 'ID', range(1, len(df_estandarizado_unique) + 1))

In [6]:
df_estandarizado_unique['NOMBRE_ESTANDARIZADO'] = df_estandarizado_unique['NOMBRE_ESTANDARIZADO'].replace('STATION WAGON', 'AUTOMOVIL')

In [7]:
df_estandarizado_unique.head()

,ID,NUMERO_IDENTIFICADOR,NOMBRE_ESTANDARIZADO,MARCA_ESTANDARIZADA
0,1,20997043,CAMION,RAM
1,2,20955958,BUS,GOLDEN DRAGON
2,3,20969631,BUS,MITSUBISHI FUSO
3,4,20987696,BUS,MITSUBISHI FUSO
4,5,20961954,AUTOMOVIL,NISSAN


In [8]:
df_estandarizado_unique.to_csv('df_endpoint_2.csv', index=False, encoding='utf-8')

In [9]:
!pip install rdflib

In [10]:
from rdflib import Graph, Namespace, Literal, RDF, XSD
import os

# Cargar el archivo CSV
csv_file = "https://deinsoluciones.cl/web_semantica/df_endpoint_2.csv"
df = pd.read_csv(csv_file, delimiter=",", encoding='utf-8')

# Crear un grafo RDF
g = Graph()

# Definir namespaces
default_ns = Namespace("http://example.org/")
xsd = Namespace("http://www.w3.org/2001/XMLSchema#")
schema = Namespace("http://schema.org/")

# Vincular namespaces
g.bind("", default_ns)
g.bind("xsd", xsd)
g.bind("schema", schema)

# Agregar datos al grafo RDF
for _, row in df.iterrows():
    marca_importacion_id = f"marca_importacion{row['ID']}"
    importacion_uri = default_ns[marca_importacion_id]

    # Definir triples RDF
    g.add((importacion_uri, RDF.type, default_ns.MarcasImportaciones))
    g.add((importacion_uri, schema.numero_identificador, Literal(int(row['NUMERO_IDENTIFICADOR']), datatype=XSD.integer)))
    g.add((importacion_uri, schema.nombre, Literal(row['NOMBRE_ESTANDARIZADO'], datatype=XSD.string)))
    g.add((importacion_uri, schema.marca, Literal(row['MARCA_ESTANDARIZADA'], datatype=XSD.string)))

# Guardar el grafo temporalmente en formato Turtle
temp_file = "marcas_importaciones_temp.ttl"
g.serialize(destination=temp_file, format="turtle")

# Leer y ajustar el contenido del archivo para modificar los prefijos
with open(temp_file, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Cabecera personalizada con prefijos
desired_prefixes = """\
prefix :       <http://example.org/>
prefix xsd:    <http://www.w3.org/2001/XMLSchema#>
prefix rdf:    <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix schema: <http://schema.org/>
"""

# Procesar líneas para corregir el formato
processed_lines = []
for line in lines:
    if not line.startswith("@prefix"):  # Ignorar los prefijos generados automáticamente
        line = line.replace(" a ", " rdf:type ")  # Asegurar que 'a' se convierte en 'rdf:type'
        line = line.replace("schema1:", "schema:")  # Corregir el prefijo incorrecto
        processed_lines.append(line)

# Guardar el archivo RDF final
output_file = "marcas_importaciones.ttl"
with open(output_file, 'w', encoding='utf-8') as file:
    file.write(desired_prefixes + "\n")
    file.writelines(processed_lines)
# Eliminar el archivo temporal
os.remove(temp_file)

print(f"Archivo RDF generado con éxito: '{output_file}'")

Archivo RDF generado con éxito: 'marcas_importaciones.ttl'
